# PFA

## Tutorial 1: small model in Titus (Python)

https://github.com/opendatagroup/hadrian/wiki/Tutorial-Small-Model-Titus

### Preparation

#### Cloned Hadrian repo

```
peterbryzgalov@peters-imac:~/work/ML/PFA
$ git clone https://github.com/opendatagroup/hadrian.git
```

#### Started Docker container
```
peterbryzgalov@peters-imac:~/work/ML/PFA
$ docker run -d -p 8888:8888 -v $(pwd):/root --name pfa pyotr777/jupyter
```


#### Installed Titus

```
root@c9a98db446e2:~/hadrian/titus# python setup.py install
```

#### Downloaded dataset

```
root@c9a98db446e2:~# wget https://github.com/opendatagroup/hadrian/wiki/data/iris.csv
```

In [3]:
import titus.version
print titus.version.__version__

0.8.4


In [4]:
from titus.genpy import PFAEngine

In [5]:
pfaDocument = {
    "input": {"type": "record",
              "name": "Iris",
              "fields": [
                  {"name": "sepal_length_cm", "type": "double"},
                  {"name": "sepal_width_cm", "type": "double"},
                  {"name": "petal_length_cm", "type": "double"},
                  {"name": "petal_width_cm", "type": "double"},
                  {"name": "class", "type": "string"}
              ]},
    "output": "string",
    "action": [
        {"if": {"<": ["input.petal_length_cm", 2.5]},
         "then": {"string": "Iris-setosa"},
         "else":
             {"if": {"<": ["input.petal_length_cm", 4.8]},
              "then": {"string": "Iris-versicolor"},
              "else":
                  {"if": {"<": ["input.petal_width_cm", 1.7]},
                  "then": {"string": "Iris-versicolor"},
                  "else": {"string": "Iris-virginica"}}
             }
        }
    ]}
engine, = PFAEngine.fromJson(pfaDocument)

In [25]:
# Test engine. Predict class for given data.
engine.action({"sepal_length_cm": 5.1, "sepal_width_cm": 3.5,
               "petal_length_cm": 2.5, "petal_width_cm": 0.2,
               "class": "-"})

'Iris-versicolor'

In [26]:
# Test engine predictions on the dataset from csv file
import csv

dataset = csv.reader(open("iris.csv"))
fields = dataset.next()
#print fields

numCorrect = 0.0
numTotal = 0.0
for datum in dataset:
    asRecord = dict(zip(fields, datum))
    guess = engine.action(asRecord)
    #print asRecord, "guessed:", guess

    if  guess == asRecord["class"]:
        numCorrect += 1.0
        print "😀",
    else:
        print "😰",
    numTotal += 1.0
print
print "accuracy", numCorrect/numTotal

😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😰 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀
accuracy 0.953333333333


### Use prettypfa to build a model

In [29]:
import titus.prettypfa

pfaDocument = titus.prettypfa.jsonNode('''
input: record(sepal_length_cm: double,
              sepal_width_cm: double,
              petal_length_cm: double,
              petal_width_cm: double)
output: string
action:
  if (input.petal_length_cm < 2.5)
    "Iris-setosa"
  else if (input.petal_length_cm < 4.8)
    "Iris-versicolor"
  else if (input.petal_width_cm < 1.7)
    "Iris-versicolor"
  else
    "Iris-virginica"
''')
engine_pfa, = PFAEngine.fromJson(pfaDocument)

In [30]:
dataset = csv.reader(open("iris.csv"))
fields = dataset.next()
#print fields

numCorrect = 0.0
numTotal = 0.0
for datum in dataset:
    asRecord = dict(zip(fields, datum))
    guess = engine_pfa.action(asRecord)
    if  guess == asRecord["class"]:
        numCorrect += 1.0
        print "😀",
    else:
        print "😰",
    numTotal += 1.0
print
print "accuracy", numCorrect/numTotal

😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😰 😀 😀 😀 😰 😰 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀 😀
accuracy 0.953333333333
